### Imports

In [34]:
import cv2
import numpy as np
from pathlib import Path
from dataclasses import dataclass, field
import random
import shutil
import concurrent.futures
import os
import shutil
import tkinter as tk
import ctypes
import platform
import tensorflow as tf
import gc
from tensorflow.keras.models import load_model


### Alte Patches, Bilder, Heatmaps und Results löschen

In [35]:
folders = ['test_picture', 'test_patches', 'heatmaps_output', 'final_results', 'final_results_decoded']

for folder in folders:
    if not os.path.exists(folder):
        print(f"Folder does not exist: {folder}")
        continue

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Delete file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete folder
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

### Bildvorverarbeitung

In [36]:

@dataclass
class MultiScaleConfig:
    patch_size: int = 256
    scale_divisors: list = field(default_factory=lambda: [1.5, 2.0, 3.0, 4.0, 6.0])
    overlap: float = 0.5 
    use_relative_min_size: bool = True
    min_relative_factor: float = 0.15
    absolute_pixel_floor: int = 128
    interpolation: int = cv2.INTER_AREA # Schnell & gut für Verkleinerung
    show_visualization: bool = False  # <--- HIER UMSCHALTEN

# --- Optimierte Kern-Logik ---

def process_single_image(img_file, cfg, out_patches_root, is_visual=True):
    """
    Verarbeitet ein Bild. Wenn is_visual=True, wird ein Vorschaubild erzeugt.
    """
    img = cv2.imread(str(img_file))
    if img is None: return [], None

    h_orig, w_orig = img.shape[:2]
    base_size = min(h_orig, w_orig)
    limit_size = max(int(base_size * cfg.min_relative_factor), cfg.absolute_pixel_floor) if cfg.use_relative_min_size else 256
    
    img_patch_dir = out_patches_root / img_file.stem
    img_patch_dir.mkdir(parents=True, exist_ok=True)
    
    vis_img = img.copy() if is_visual else None
    colors = [(255, 0, 0), (255, 255, 0), (0, 255, 0), (0, 255, 255), (0, 165, 255)]
    
    local_metadata = []
    patch_count = 0

    for scale_idx, divisor in enumerate(cfg.scale_divisors):
        win_size = max(int(base_size / divisor), limit_size)
        win_size = min(win_size, base_size)
        stride = max(1, int(win_size * (1 - cfg.overlap)))
        
        # Einmaliges Padding pro Skalierung spart massiv Zeit
        # Wir padden so viel, dass wir beim Slicing keine Fehler bekommen
        pad = win_size 
        img_padded = cv2.copyMakeBorder(img, 0, pad, 0, pad, cv2.BORDER_CONSTANT, value=[0,0,0])

        for y in range(0, h_orig - win_size + stride, stride):
            for x in range(0, w_orig - win_size + stride, stride):
                # Slice direkt aus dem gepaddeten Bild (blitzschnell)
                patch = img_padded[y : y + win_size, x : x + win_size]
                
                if patch.shape[0] != cfg.patch_size:
                    patch = cv2.resize(patch, (cfg.patch_size, cfg.patch_size), interpolation=cfg.interpolation)
                
                patch_name = f"S{scale_idx}_p{patch_count}.jpg"
                cv2.imwrite(str(img_patch_dir / patch_name), patch)
                
                local_metadata.append(f"{img_file.name};{img_file.stem}/{patch_name};{x};{y};{win_size};{w_orig};{h_orig}")
                
                if is_visual:
                    color = colors[scale_idx % len(colors)]
                    cv2.rectangle(vis_img, (x, y), (x + win_size, y + win_size), color, max(1, int(win_size/200)))
                
                patch_count += 1
                
    return local_metadata, vis_img

def main():
    # --- PFADE ---
    input_dir = Path("~/DatenUbuntu/Studium/1.Semester/KI-Projekt/modeltest/all_pictures").expanduser()
    test_picture_dir = Path("test_picture") 
    output_root = Path("test_patches")    
    cfg = MultiScaleConfig()

    # Vorbereitung
    if test_picture_dir.exists(): shutil.rmtree(test_picture_dir)
    if output_root.exists(): shutil.rmtree(output_root)
    test_picture_dir.mkdir(parents=True, exist_ok=True)
    out_patches_root = output_root / "patches"
    out_patches_root.mkdir(parents=True, exist_ok=True)

    img_files = list(input_dir.glob("*.[jJ][pP][gG]")) + list(input_dir.glob("*.[pP][nN][gG]"))
    if not img_files: return

    user_input = input(f"Bilder (Zahl oder 'all' [Gesamt: {len(img_files)}]): ")
    num = len(img_files) if user_input.lower() == 'all' else int(user_input)
    selected_files = random.sample(img_files, min(num, len(img_files)))

    all_patch_metadata = []

    # --- MODUS ENTSCHEIDUNG ---
    if cfg.show_visualization:
        print("📺 Visueller Modus (Sequenziell)... 'q' zum Abbrechen.")
        for f in selected_files:
            meta, vis = process_single_image(f, cfg, out_patches_root, is_visual=True)
            all_patch_metadata.extend(meta)
            shutil.copy2(f, test_picture_dir / f.name)
            
            # Anzeige
            h, w = vis.shape[:2]
            scale = 800 / max(h, w)
            cv2.imshow("Preview", cv2.resize(vis, (int(w*scale), int(h*scale))))
            print(f"✅ {f.name} - {len(meta)} Patches")
            if cv2.waitKey(1) & 0xFF == ord('q'): break
        cv2.destroyAllWindows()
    
    else:
        print(f"🚀 Turbo-Modus: Parallele Verarbeitung auf {os.cpu_count()} Kernen...")
        with concurrent.futures.ProcessPoolExecutor() as executor:
            futures = {executor.submit(process_single_image, f, cfg, out_patches_root, False): f for f in selected_files}
            for future in concurrent.futures.as_completed(futures):
                f = futures[future]
                meta, _ = future.result()
                all_patch_metadata.extend(meta)
                shutil.copy2(f, test_picture_dir / f.name)
                print(f"✅ {f.name} - {len(meta)} Patches")

    # Metadaten speichern
    with open(output_root / "metadata.txt", "w") as f:
        f.write("\n".join(all_patch_metadata))
    
    print(f"\nFertig! Patches in '{output_root}' gespeichert.")

if __name__ == "__main__":
    main()

🚀 Turbo-Modus: Parallele Verarbeitung auf 16 Kernen...
✅ CKlasse_Bilderreihe 3_P_IMG_9546_rot4.jpg - 247 Patches
✅ Seat_P_45.jpg - 291 Patches
✅ CKlasse_Bilderreihe 2_P_IMG_9443_rot-4.jpg - 247 Patches
✅ Passat_P3_frame800.jpg - 388 Patches
✅ Passat_N2_frame2530.jpg - 388 Patches
✅ Passat_P1_frame1430.jpg - 388 Patches
✅ Passat_N1_frame1680.jpg - 388 Patches
✅ Focus_P1_frame_530.jpg - 388 Patches
✅ Passat_P3_frame1770.jpg - 388 Patches
✅ GKLASSE_ALT_P1_frame1800.jpg - 388 Patches
✅ FocusP3_frame_1540.jpg - 388 Patches
✅ Passat_P2_frame400.jpg - 388 Patches
✅ Passat_P2_frame1330.jpg - 388 Patches
✅ Focus_P1_frame_3720.jpg - 388 Patches
✅ Golf_IV_N_45.jpg - 291 Patches
✅ CKlasse_Bilderreihe 4_P_IMG_9651_rot14.jpg - 247 Patches
✅ Focus_P2_frame_3000.jpg - 388 Patches
✅ Passat_N2_frame590.jpg - 388 Patches
✅ GKLASSE_NEU_N2_frame2790.jpg - 388 Patches
✅ Passat_N1_frame1550.jpg - 388 Patches
✅ FocusP3_frame_100.jpg - 388 Patches
✅ GKLASSE_NEU_P1_frame3280.jpg - 388 Patches
✅ Passat_P1_frame2

### Modell

In [37]:

# --- GPU Setup ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' # Falls nötig aktivieren

def setup_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU Beschleunigung aktiv.")
    except: pass
setup_gpu()

class QRAllInOneConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    original_img_dir = Path('test_picture')
    
    # Ausgabe-Ordner
    output_dir_final = Path('final_results')
    output_dir_heat = Path('heatmaps_output')
    
    # Parameter
    min_vote_prob = 0.3    # Ab wann zählt ein Patch zur Heatmap?
    vote_threshold = 5     # Heatmap-Wert für "Box zeichnen"
    fixed_vis_max = 25.0   # Skalierungswert für die Heatmap-PNG (für den Viewer)
    
    start_batch_size = 32 

def run_pipeline():
    cfg = QRAllInOneConfig()
    cfg.output_dir_final.mkdir(exist_ok=True)
    cfg.output_dir_heat.mkdir(exist_ok=True) # Heatmap Ordner erstellen
    
    if not cfg.metadata_file.exists():
        print("❌ Metadaten-Datei nicht gefunden!")
        return
    print("⏳ Lade Modell...")
    model = load_model(cfg.model_path, compile=False)
    
    # Metadaten einlesen
    with open(cfg.metadata_file, "r") as f:
        lines = [l.strip().split(";") for l in f.readlines()]

    images_dict = {}
    for img_name, rel, px, py, ps, w_orig, h_orig in lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({"path": rel, "x": int(px), "y": int(py), "s": int(ps)})

    print(f"--- Starte Kombi-Pipeline (Heatmaps + Boxen) ---")

    for img_name, info in images_dict.items():
        orig_img = cv2.imread(str(cfg.original_img_dir / img_name))
        if orig_img is None: continue
        
        h_orig, w_orig = info["h"], info["w"]
        patch_list = info["patches"]
        
        # --- Batch-Verarbeitung ---
        all_patch_imgs = []
        for p in patch_list:
            p_img = cv2.imread(str(cfg.patch_folder / p["path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256))
            all_patch_imgs.append(p_img.astype(np.float32))

        if not all_patch_imgs: continue
        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)

        # Vorhersage
        preds = None
        current_bs = cfg.start_batch_size
        while current_bs >= 1:
            try:
                preds = model.predict(input_batch, batch_size=current_bs, verbose=0)
                break 
            except tf.errors.ResourceExhaustedError:
                current_bs //= 2
                gc.collect()
        
        if preds is None: continue

        # --- Heatmap Berechnung ---
        heatmap_sum = np.zeros((h_orig, w_orig), dtype=np.float32)
        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            if prob > cfg.min_vote_prob:
                p = patch_list[i]
                heatmap_sum[p["y"]:p["y"]+p["s"], p["x"]:p["x"]+p["s"]] += prob

        # ---------------------------------------------------------
        # SCHRITT 1: Heatmap speichern (für den Viewer)
        # ---------------------------------------------------------
        # Normierung genau wie im Viewer erwartet (auf Basis von 25.0)
        heatmap_norm = (heatmap_sum / cfg.fixed_vis_max) * 255.0
        heatmap_8bit = np.clip(heatmap_norm, 0, 255).astype(np.uint8)
        
        heat_out_path = cfg.output_dir_heat / f"{Path(img_name).stem}_heatmap.png"
        cv2.imwrite(str(heat_out_path), heatmap_8bit)

        # ---------------------------------------------------------
        # SCHRITT 2: Boxen zeichnen (für das Ergebnisbild)
        # ---------------------------------------------------------
        _, thresh = cv2.threshold(heatmap_sum, cfg.vote_threshold, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        result_img = orig_img.copy()
        found_count = 0
        
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 30 and h > 30: 
                cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 255, 0), 4)
                # cv2.putText(result_img, "QR", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                found_count += 1

        final_out_path = cfg.output_dir_final / f"final_{img_name}"
        cv2.imwrite(str(final_out_path), result_img)
        
        print(f"✅ {img_name}: {found_count} Objekte | Heatmap & Bild gespeichert.")

        del input_batch, all_patch_imgs, preds, heatmap_sum
        gc.collect()

    tf.keras.backend.clear_session()
    print(f"\nFertig! Viewer kann jetzt gestartet werden.")

if __name__ == "__main__":
    run_pipeline()

⏳ Lade Modell...
--- Starte Kombi-Pipeline (Heatmaps + Boxen) ---
✅ CKlasse_Bilderreihe 3_P_IMG_9546_rot4.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ Seat_P_45.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ CKlasse_Bilderreihe 2_P_IMG_9443_rot-4.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ Passat_P3_frame800.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ Passat_N2_frame2530.jpg: 0 Objekte | Heatmap & Bild gespeichert.
✅ Passat_P1_frame1430.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ Passat_N1_frame1680.jpg: 0 Objekte | Heatmap & Bild gespeichert.
✅ Focus_P1_frame_530.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ Passat_P3_frame1770.jpg: 2 Objekte | Heatmap & Bild gespeichert.
✅ GKLASSE_ALT_P1_frame1800.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ FocusP3_frame_1540.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ Passat_P2_frame400.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ Passat_P2_frame1330.jpg: 1 Objekte | Heatmap & Bild gespeichert.
✅ Focus_P1_frame_3720.jpg: 1 Objekt

### Visualisierung

In [38]:
# --- DPI FIX FÜR WINDOWS ---
if platform.system() == "Windows":
    try:
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        ctypes.windll.user32.SetProcessDPIAware()

# --- 1. DESIGN & STANDARDS ---
TARGET_HEIGHT = 700  # Basis-Höhe für die Einzelbilder
HEADER_HEIGHT = 60
FOOTER_HEIGHT = 130
MARGIN = 20
FIXED_MAX_SCORE = 25.0 

C_BG = (24, 24, 27)
C_HEADER = (39, 39, 42)
C_ACCENT = (14, 165, 233)
C_TEXT = (228, 228, 231)
C_SUCCESS = (34, 197, 94)
C_FAIL = (239, 68, 68)

path_original = Path('test_picture')
path_heatmap = Path('heatmaps_output')
path_decision = Path('final_results')

def get_screen_size():
    root = tk.Tk()
    sw = root.winfo_screenwidth()
    sh = root.winfo_screenheight()
    root.destroy()
    return sw, sh

def resize_with_aspect(img, target_h):
    h, w = img.shape[:2]
    scale = target_h / h
    return cv2.resize(img, (int(w * scale), target_h), interpolation=cv2.INTER_AREA)

def show_interactive_evaluation():
    img_files = list(path_original.glob("*.[jJ][pP][gG]")) + list(path_original.glob("*.[pP][nN][gG]"))
    if not img_files:
        print("❌ Keine Bilder gefunden!")
        return
    
    random.shuffle(img_files)

    # Bildschirmmaße abrufen und Puffer lassen
    sw, sh = get_screen_size()
    print(f"Detected Screen Resolution: {sw}x{sh}")
    
    # Maximale Fenstergröße (90% der Breite, 80% der Höhe für Taskleiste/Fensterrahmen)
    max_w = int(sw * 0.90)
    max_h = int(sh * 0.85)

    stats = {"richtig": 0, "falsch": 0, "gesamt": 0}

    for idx, img_file in enumerate(img_files):
        # Daten laden
        img_orig = cv2.imread(str(img_file))
        heat_path = path_heatmap / f"{img_file.stem}_heatmap.png"
        img_heat_gray = cv2.imread(str(heat_path), cv2.IMREAD_GRAYSCALE) if heat_path.exists() else None
        img_final = cv2.imread(str(path_decision / f"final_{img_file.name}"))

        if img_orig is None or img_final is None: continue

        # Score Logik
        max_pixel_val = np.max(img_heat_gray) if img_heat_gray is not None else 0
        real_score = (max_pixel_val / 255.0) * FIXED_MAX_SCORE
        qr_found = real_score >= 5.0 

        # Einzelbilder skalieren
        res_orig = resize_with_aspect(img_orig, TARGET_HEIGHT)
        res_final = resize_with_aspect(img_final, TARGET_HEIGHT)
        if img_heat_gray is not None:
            res_heat_color = cv2.applyColorMap(resize_with_aspect(img_heat_gray, TARGET_HEIGHT), cv2.COLORMAP_JET)
        else:
            res_heat_color = np.zeros_like(res_orig)

        # Canvas berechnen
        content_w = res_orig.shape[1] + res_heat_color.shape[1] + res_final.shape[1] + (2 * MARGIN)
        content_h = HEADER_HEIGHT + TARGET_HEIGHT + FOOTER_HEIGHT
        
        canvas = np.full((content_h, content_w, 3), C_BG, dtype=np.uint8)

        # UI Elemente zeichnen (Header, Bilder, Footer)
        canvas[0:HEADER_HEIGHT, :] = C_HEADER
        curr_x = 0
        for title, w in [("ORIGINAL", res_orig.shape[1]), ("VOTE DENSITY", res_heat_color.shape[1]), ("FINAL", res_final.shape[1])]:
            cv2.putText(canvas, title, (curr_x + 10, 40), cv2.FONT_HERSHEY_DUPLEX, 0.6, C_ACCENT, 1, cv2.LINE_AA)
            curr_x += w + MARGIN

        canvas[HEADER_HEIGHT:HEADER_HEIGHT+TARGET_HEIGHT, 0:res_orig.shape[1]] = res_orig
        x_off = res_orig.shape[1] + MARGIN
        canvas[HEADER_HEIGHT:HEADER_HEIGHT+TARGET_HEIGHT, x_off:x_off+res_heat_color.shape[1]] = res_heat_color
        x_off += res_heat_color.shape[1] + MARGIN
        canvas[HEADER_HEIGHT:HEADER_HEIGHT+TARGET_HEIGHT, x_off:x_off+res_final.shape[1]] = res_final

        # Footer Texte
        f_y = HEADER_HEIGHT + TARGET_HEIGHT
        cv2.line(canvas, (0, f_y), (content_w, f_y), (63, 63, 70), 2)
        cv2.putText(canvas, f"FILE: {img_file.name} | {idx+1}/{len(img_files)}", (20, f_y + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (161, 161, 170), 1, cv2.LINE_AA)
        
        status_color = C_SUCCESS if qr_found else C_FAIL
        cv2.putText(canvas, "QR DETECTED" if qr_found else "NO DETECTION", (content_w - 320, f_y + 50), cv2.FONT_HERSHEY_DUPLEX, 0.7, status_color, 2, cv2.LINE_AA)
        cv2.putText(canvas, f"SCORE: {real_score:.2f}", (content_w - 320, f_y + 85), cv2.FONT_HERSHEY_SIMPLEX, 0.6, C_TEXT, 1, cv2.LINE_AA)
        cv2.putText(canvas, "[J] CORRECT   [N] WRONG   [Q] EXIT", (int(content_w/2) - 150, f_y + 110), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (113, 113, 122), 1, cv2.LINE_AA)

        # --- SKALIERUNG AUF BILDSCHIRMGRÖSSE ---
        final_canvas = canvas
        if content_w > max_w or content_h > max_h:
            scale = min(max_w / content_w, max_h / content_h)
            final_canvas = cv2.resize(canvas, (int(content_w * scale), int(content_h * scale)), interpolation=cv2.INTER_AREA)

        # --- FENSTER ANZEIGEN & ZENTRIEREN ---
        win_name = "AI Evaluation Dashboard"
        actual_w, actual_h = final_canvas.shape[1], final_canvas.shape[0]
        
        cv2.namedWindow(win_name, cv2.WINDOW_NORMAL) # WINDOW_NORMAL erlaubt Skalierung
        cv2.resizeWindow(win_name, actual_w, actual_h)
        cv2.moveWindow(win_name, (sw - actual_w) // 2, (sh - actual_h) // 2)
        
        cv2.imshow(win_name, final_canvas)
        
        key = cv2.waitKey(0) & 0xFF
        if key == ord('q'): break
        elif key == ord('j'): stats["richtig"] += 1; stats["gesamt"] += 1
        elif key == ord('n'): stats["falsch"] += 1; stats["gesamt"] += 1

    cv2.destroyAllWindows()
    if stats["gesamt"] > 0:
        print(f"\nFinal Accuracy: {(stats['richtig']/stats['gesamt']*100):.2f}%")

if __name__ == "__main__":
    show_interactive_evaluation()

Detected Screen Resolution: 1920x1200

Final Accuracy: 99.09%


### QR-Code encoding

In [39]:
import cv2
import numpy as np
from pathlib import Path
from pyzbar.pyzbar import decode

# ==========================================
# 1. KONFIGURATION & SETUP
# ==========================================
class PostProcessConfig:
    original_img_dir = Path('test_picture')
    heatmap_dir = Path('heatmaps_output')
    
    base_output_dir = Path('final_results_decoded')
    dir_success = base_output_dir / 'success'
    dir_failed = base_output_dir / 'failed'
    
    log_file = base_output_dir / 'scan_results.txt'
    
    heatmap_threshold_pixel_val = 51 
    padding = 25  

# ==========================================
# 2. FUNKTION: ERWEITERTE DEKODIERUNG
# ==========================================
def apply_decoding_tricks(roi):
    """
    Versucht aggressiv, einen Code in der ROI zu finden (Graustufen, Threshold, Zoom, Rotation).
    """
    attempts = []
    
    # A: Graustufen
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    attempts.append(gray)
    
    # B: Otsu Binarisierung (Starker Kontrast)
    _, binary_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    attempts.append(binary_otsu)
    
    # C: Adaptives Thresholding (Gegen Schatten)
    adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    attempts.append(adaptive)

    # D: Zoom & Schärfen
    roi_big = cv2.resize(gray, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    roi_sharp = cv2.filter2D(roi_big, -1, kernel)
    attempts.append(roi_sharp)

    for img_variant in attempts:
        # 1. Normaler Versuch
        res = decode(img_variant)
        if res: return res[0]
        
        # 2. Rotations-Versuche (30°, 45°, 60° in beide Richtungen)
        h, w = img_variant.shape
        center = (w // 2, h // 2)
        
        for angle in [ 15, 30, 45, 60, 90, -15 ,-30, -45, -60, -90]: 
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated = cv2.warpAffine(img_variant, M, (w, h))
            res_rot = decode(rotated)
            if res_rot: return res_rot[0]
            
    return None

# ==========================================
# 3. HAUPTPROGRAMM
# ==========================================
def run_decoding_optimized():
    cfg = PostProcessConfig()
    
    cfg.dir_success.mkdir(parents=True, exist_ok=True)
    cfg.dir_failed.mkdir(parents=True, exist_ok=True)
    
    heatmap_files = list(cfg.heatmap_dir.glob("*_heatmap.png"))
    if not heatmap_files:
        print("❌ Keine Heatmaps gefunden!")
        return

    print(f"📂 Starte Analyse von {len(heatmap_files)} Heatmaps...")
    
    # --- STATISTIK ZÄHLER INITIALISIEREN ---
    stats_processed_images = 0  # Wie viele Bilder wurden tatsächlich bearbeitet (Original gefunden)
    stats_success = 0           # Bilder, in denen mind. 1 Code gefunden wurde
    stats_failed = 0            # Bilder, in denen nichts gefunden wurde
    stats_total_codes = 0       # Gesamtzahl aller gefundenen Codes (kann > stats_success sein)

    with open(cfg.log_file, 'w', encoding='utf-8') as log:
        log.write("Dateiname;Status;Inhalt\n") 
        
        for hm_file in heatmap_files:
            stem = hm_file.stem.replace("_heatmap", "")
            
            # Originalbild suchen
            orig_path = None
            for ext in [".jpg", ".jpeg", ".png", ".JPG", ".PNG"]:
                possible = cfg.original_img_dir / (stem + ext)
                if possible.exists():
                    orig_path = possible
                    break
            
            if not orig_path: continue

            img_orig = cv2.imread(str(orig_path))
            img_heatmap = cv2.imread(str(hm_file), cv2.IMREAD_GRAYSCALE)
            
            if img_orig is None: continue
            
            # Zähler für bearbeitete Bilder erhöhen
            stats_processed_images += 1
            
            h_orig, w_orig = img_orig.shape[:2]

            # Heatmap verarbeiten
            _, thresh = cv2.threshold(img_heatmap, cfg.heatmap_threshold_pixel_val, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            result_img = img_orig.copy()
            image_has_decoded_code = False
            found_contents = []

            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                if w < 30 or h < 30: continue
                
                # ROI ausschneiden
                x_pad = max(0, x - cfg.padding)
                y_pad = max(0, y - cfg.padding)
                w_pad = min(w_orig - x_pad, w + 2*cfg.padding)
                h_pad = min(h_orig - y_pad, h + 2*cfg.padding)
                
                roi = img_orig[y_pad : y_pad + h_pad, x_pad : x_pad + w_pad]
                
                decoded_obj = apply_decoding_tricks(roi)

                if decoded_obj:
                    image_has_decoded_code = True
                    content = decoded_obj.data.decode("utf-8")
                    type_code = decoded_obj.type
                    
                    found_contents.append(content)
                    stats_total_codes += 1 # Globalen Code-Zähler erhöhen
                    
                    print(f"   ✅ {stem} [{type_code}]: {content}")
                    
                    cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 255, 0), 4)
                    cv2.putText(result_img, content[:15], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                else:
                    cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 0, 255), 2)

            # --- Speichern & Zählen ---
            output_filename = f"res_{orig_path.name}"
            
            if image_has_decoded_code:
                # Statistik: Ein Erfolg
                stats_success += 1
                
                cv2.imwrite(str(cfg.dir_success / output_filename), result_img)
                for c in found_contents:
                    log.write(f"{orig_path.name};OK;{c}\n")
            else:
                # Statistik: Ein Misserfolg
                stats_failed += 1
                
                if len(contours) > 0:
                    cv2.imwrite(str(cfg.dir_failed / output_filename), result_img)
                    log.write(f"{orig_path.name};FAILED;-\n")

    # ==========================================
    # 4. ABSCHLUSS & STATISTIK AUSGABE
    # ==========================================
    print("\n" + "="*50)
    print("             ERGEBNIS ZUSAMMENFASSUNG")
    print("="*50)
    
    if stats_processed_images > 0:
        success_rate = (stats_success / stats_processed_images) * 100
        fail_rate = (stats_failed / stats_processed_images) * 100
    else:
        success_rate = 0
        fail_rate = 0

    print(f"Bilder verarbeitet:      {stats_processed_images}")
    print(f"Bilder mit Fund (OK):    {stats_success} ({success_rate:.1f}%)")
    print(f"Bilder ohne Fund (Fail): {stats_failed} ({fail_rate:.1f}%)")
    print("-" * 50)
    print(f"Anzahl gelesener Codes:  {stats_total_codes}")
    print("="*50)
    print(f"Logfile: {cfg.log_file}")
    print(f"Bilder gespeichert in: {cfg.base_output_dir}")

if __name__ == "__main__":
    run_decoding_optimized()

📂 Starte Analyse von 220 Heatmaps...
   ✅ Passat_P1_frame40 [QRCODE]: https://testsigma.com/
   ✅ GKLASSE_NEU_P1_frame4050 [QRCODE]: https://testsigma.com/
   ✅ CKlasse_Bilderreihe 5_P_IMG_9517_rot10 [QRCODE]: https://www.h-ka.de/
   ✅ Focus_P2_frame_250 [QRCODE]: https://testsigma.com/
   ✅ GKLASSE_NEU_P1_frame390 [QRCODE]: https://testsigma.com/
   ✅ Passat_P1_frame1900 [QRCODE]: https://testsigma.com/
   ✅ FocusP3_frame_1590 [QRCODE]: https://testsigma.com/
   ✅ Passat_P2_frame2760 [QRCODE]: https://testsigma.com/


	i=13 f=-1(001) part=1


   ✅ Focus_P2_frame_2460 [QRCODE]: https://testsigma.com/
   ✅ FocusP3_frame_2370 [QRCODE]: https://testsigma.com/
   ✅ 20251027_161715 [QRCODE]: https://www.h-ka.de/
   ✅ FocusP4_frame_2060 [QRCODE]: https://testsigma.com/
   ✅ FocusP3_frame_320 [QRCODE]: https://testsigma.com/
   ✅ FocusP3_frame_1050 [QRCODE]: https://testsigma.com/
   ✅ Golf_IV_P_19 [QRCODE]: U84 06548734 002
   ✅ FocusP3_frame_3110 [QRCODE]: https://testsigma.com/
   ✅ Passat_P1_frame3080 [QRCODE]: https://testsigma.com/


	i=1 f=-1(000) part=0


   ✅ CKlasse_Bilderreihe 5_P_IMG_9511_contrast [QRCODE]: https://www.h-ka.de/
   ✅ Focus_P2_frame_0 [QRCODE]: https://testsigma.com/
   ✅ FocusP3_frame_1520 [QRCODE]: https://testsigma.com/
   ✅ IMG_2908 [CODE128]: 
   ✅ 20251213_125143_039 [CODE128]: 
   ✅ Passat_P1_frame300 [QRCODE]: https://testsigma.com/
   ✅ CKlasse_Bilderreihe 2_P_IMG_9423_original [QRCODE]: https://www.h-ka.de/
   ✅ Passat_P1_frame120 [QRCODE]: https://testsigma.com/
   ✅ Focus_P2_frame_40 [QRCODE]: https://testsigma.com/
   ✅ Focus_P2_frame_1260 [QRCODE]: https://testsigma.com/
   ✅ Passat_P1_frame3060 [QRCODE]: https://testsigma.com/
   ✅ FocusP3_frame_1540 [QRCODE]: https://testsigma.com/
   ✅ IMG_6779 [QRCODE]: https://www.h-ka.de
   ✅ Focus_P1_frame_2560 [QRCODE]: https://testsigma.com/


	i=29 f=-1(001) part=1


   ✅ GKLASSE_NEU_P1_frame3750 [QRCODE]: https://testsigma.com/
   ✅ FocusP4_frame_330 [QRCODE]: https://testsigma.com/
   ✅ FocusP4_frame_3030 [QRCODE]: https://testsigma.com/
   ✅ Passat_P1_frame2640 [QRCODE]: https://testsigma.com/
   ✅ GKLASSE_NEU_P1_frame3280 [QRCODE]: https://testsigma.com/


	i=19 f=-1(000) part=0
	i=26 f=-1(000) part=0



             ERGEBNIS ZUSAMMENFASSUNG
Bilder verarbeitet:      220
Bilder mit Fund (OK):    36 (16.4%)
Bilder ohne Fund (Fail): 184 (83.6%)
--------------------------------------------------
Anzahl gelesener Codes:  36
Logfile: final_results_decoded/scan_results.txt
Bilder gespeichert in: final_results_decoded


## test

In [40]:
import cv2
import numpy as np
from pathlib import Path
from pyzbar.pyzbar import decode, ZBarSymbol

# ==========================================
# 1. KONFIGURATION
# ==========================================
class PostProcessConfig:
    original_img_dir = Path('test_picture')
    heatmap_dir = Path('heatmaps_output')
    
    base_output_dir = Path('final_results_back_to_roots')
    dir_success = base_output_dir / 'success'
    dir_failed = base_output_dir / 'failed'
    log_file = base_output_dir / 'scan_results.txt'
    
    heatmap_threshold_pixel_val = 51  
    padding = 25  

# ==========================================
# 2. HILFSFUNKTIONEN
# ==========================================
def add_quiet_zone(img):
    """
    Fügt einen weißen Rahmen hinzu. Das ist oft der einzige Grund,
    warum ein perfekter Crop nicht gelesen wird.
    """
    border = 30
    return cv2.copyMakeBorder(img, border, border, border, border, cv2.BORDER_CONSTANT, value=[255, 255, 255])

# ==========================================
# 3. DEKODIERUNG (DEIN ORIGINAL + QUIET ZONE)
# ==========================================
def apply_decoding_tricks(roi):
    """
    Dein ursprünglicher 19%-Ansatz, aber mit erzwungener 'Quiet Zone'.
    """
    if roi is None or roi.size == 0: return None

    # SCHRITT 0: Quiet Zone hinzufügen (Das fehlte im Original)
    roi = add_quiet_zone(roi)

    attempts = []
    
    # A: Graustufen (Standard)
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    attempts.append(gray)
    
    # B: Otsu Binarisierung (Starker Kontrast)
    _, binary_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    attempts.append(binary_otsu)
    
    # C: Adaptives Thresholding (Gegen Schatten)
    adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    attempts.append(adaptive)

    # D: Zoom & Schärfen (Deine Logik)
    # 2x Zoom hilft bei kleiner Auflösung enorm
    roi_big = cv2.resize(gray, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    roi_sharp = cv2.filter2D(roi_big, -1, kernel)
    attempts.append(roi_sharp)
    
    # E: Zoom & Binarisierung (Neu kombiniert)
    # Manchmal hilft Zoom + harter Kontrast zusammen
    _, big_binary = cv2.threshold(roi_sharp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    attempts.append(big_binary)

    # --- SCHLEIFE ---
    for img_variant in attempts:
        # 1. Normaler Versuch
        res = decode(img_variant, symbols=[ZBarSymbol.QRCODE])
        if res: return res[0]
        
        # 2. Rotations-Versuche
        # Wir nutzen deine ursprüngliche Winkel-Liste, die gut funktionierte
        h, w = img_variant.shape
        center = (w // 2, h // 2)
        
        # Winkel erweitert um leichte Schieflagen (10, -10)
        for angle in [10, -10, 30, 45, 60, 90, -30, -45, -60, -90]: 
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            # WICHTIG: borderValue=255 (Weiß) statt Schwarz, damit der Rand erhalten bleibt!
            rotated = cv2.warpAffine(img_variant, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=255)
            
            res_rot = decode(rotated, symbols=[ZBarSymbol.QRCODE])
            if res_rot: return res_rot[0]
            
    return None

# ==========================================
# 4. HAUPTPROGRAMM
# ==========================================
def run_decoding_restored():
    cfg = PostProcessConfig()
    cfg.dir_success.mkdir(parents=True, exist_ok=True)
    cfg.dir_failed.mkdir(parents=True, exist_ok=True)
    
    heatmap_files = list(cfg.heatmap_dir.glob("*_heatmap.png"))
    if not heatmap_files:
        print("❌ Keine Heatmaps gefunden!")
        return

    print(f"📂 Starte Analyse (Restored 19% Version + QuietZone)...")
    
    stats_processed = 0
    stats_success = 0
    stats_failed = 0
    stats_codes = 0

    with open(cfg.log_file, 'w', encoding='utf-8') as log:
        log.write("Dateiname;Status;Inhalt\n") 
        
        for hm_file in heatmap_files:
            stem = hm_file.stem.replace("_heatmap", "")
            
            orig_path = None
            for ext in [".jpg", ".jpeg", ".png", ".JPG", ".PNG"]:
                possible = cfg.original_img_dir / (stem + ext)
                if possible.exists(): orig_path = possible; break
            
            if not orig_path: continue
            stats_processed += 1
            
            img_orig = cv2.imread(str(orig_path))
            img_heatmap = cv2.imread(str(hm_file), cv2.IMREAD_GRAYSCALE)
            
            if img_orig is None: continue
            h_orig, w_orig = img_orig.shape[:2]

            _, thresh = cv2.threshold(img_heatmap, cfg.heatmap_threshold_pixel_val, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            result_img = img_orig.copy()
            image_has_code = False
            found_contents = []

            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                if w < 20 or h < 20: continue 
                
                # ROI Cut
                x_pad = max(0, x - cfg.padding)
                y_pad = max(0, y - cfg.padding)
                w_pad = min(w_orig - x_pad, w + 2*cfg.padding)
                h_pad = min(h_orig - y_pad, h + 2*cfg.padding)
                roi = img_orig[y_pad : y_pad + h_pad, x_pad : x_pad + w_pad]
                
                decoded_obj = apply_decoding_tricks(roi)

                if decoded_obj:
                    image_has_code = True
                    content = decoded_obj.data.decode("utf-8")
                    found_contents.append(content)
                    stats_codes += 1
                    
                    print(f"   ✅ {stem}: {content}")
                    cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 255, 0), 4)
                    cv2.putText(result_img, "OK", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                else:
                    cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 0, 255), 2)

            out_name = f"res_{orig_path.name}"
            if image_has_code:
                stats_success += 1
                cv2.imwrite(str(cfg.dir_success / out_name), result_img)
                for c in found_contents:
                    log.write(f"{orig_path.name};OK;{c}\n")
            elif len(contours) > 0:
                stats_failed += 1
                cv2.imwrite(str(cfg.dir_failed / out_name), result_img)
                log.write(f"{orig_path.name};FAILED;-\n")

    print("\n" + "="*50)
    print("             ERGEBNIS (RESTORED)")
    print("="*50)
    if stats_processed > 0:
        rate = (stats_success / stats_processed) * 100
    else: rate = 0
    print(f"Verarbeitet:  {stats_processed}")
    print(f"Erfolg:       {stats_success} ({rate:.1f}%)")
    print(f"Fehlschlag:   {stats_failed}")
    print("="*50)

if __name__ == "__main__":
    run_decoding_restored()

📂 Starte Analyse (Restored 19% Version + QuietZone)...
   ✅ Passat_P1_frame40: https://testsigma.com/
   ✅ GKLASSE_NEU_P1_frame4050: https://testsigma.com/
   ✅ CKlasse_Bilderreihe 5_P_IMG_9517_rot10: https://www.h-ka.de/
   ✅ Focus_P2_frame_250: https://testsigma.com/
   ✅ GKLASSE_NEU_P1_frame390: https://testsigma.com/
   ✅ Passat_P1_frame590: https://testsigma.com/
   ✅ Passat_P1_frame1900: https://testsigma.com/
   ✅ FocusP3_frame_1590: https://testsigma.com/
   ✅ Passat_P2_frame2760: https://testsigma.com/
   ✅ FocusP3_frame_2370: https://testsigma.com/
   ✅ Passat_P1_frame2840: https://testsigma.com/
   ✅ 20251027_161715: https://www.h-ka.de/
   ✅ FocusP4_frame_2060: https://testsigma.com/
   ✅ FocusP3_frame_320: https://testsigma.com/
   ✅ FocusP3_frame_1050: https://testsigma.com/
   ✅ Golf_IV_P_19: U84 06548734 002
   ✅ FocusP3_frame_3110: https://testsigma.com/
   ✅ CKlasse_Bilderreihe 2_P_IMG_9452_rot-9: https://www.h-ka.de/
   ✅ Passat_P1_frame3080: https://testsigma.com/
 

In [41]:
import cv2
import numpy as np
from pathlib import Path
from pyzbar.pyzbar import decode, ZBarSymbol

# ==========================================
# 1. KONFIGURATION
# ==========================================
class PostProcessConfig:
    original_img_dir = Path('test_picture')
    heatmap_dir = Path('heatmaps_output')
    
    base_output_dir = Path('final_results_combined_force')
    dir_success = base_output_dir / 'success'
    dir_failed = base_output_dir / 'failed'
    log_file = base_output_dir / 'scan_results.txt'
    
    # Dein bevorzugter Wert
    heatmap_threshold_pixel_val = 51  
    padding = 60

# ==========================================
# 2. DER "20%-DECODER" (Dein Sieger-Code)
# ==========================================
def add_quiet_zone(img):
    """Fügt den lebenswichtigen weißen Rand hinzu."""
    border = 30
    return cv2.copyMakeBorder(img, border, border, border, border, cv2.BORDER_CONSTANT, value=[255, 255, 255])

def robust_decode_func(roi_chunk):
    """
    Das ist EXAKT die Logik aus deiner 20%-Lösung.
    Wir haben sie in eine Funktion gepackt, um sie mehrfach anwenden zu können.
    """
    if roi_chunk is None or roi_chunk.size == 0: return None

    # 1. Quiet Zone (Basis für alles)
    roi = add_quiet_zone(roi_chunk)
    
    attempts = []
    
    # A: Graustufen
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    attempts.append(gray)
    
    # B: Otsu
    _, binary_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    attempts.append(binary_otsu)
    
    # C: Adaptive Threshold
    adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    attempts.append(adaptive)

    # D: Zoom & Schärfen (Das fehlte im 16% Versuch!)
    roi_big = cv2.resize(gray, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    roi_sharp = cv2.filter2D(roi_big, -1, kernel)
    attempts.append(roi_sharp)
    
    # E: Zoom & Binarisierung
    _, big_binary = cv2.threshold(roi_sharp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    attempts.append(big_binary)

    # --- DIE SUCHE ---
    for img_variant in attempts:
        # 1. Scan normal
        res = decode(img_variant, symbols=[ZBarSymbol.QRCODE])
        if res: return res[0]
        
        # 2. Rotation (Nur wenn Bildgröße sinnvoll ist, um Zeit zu sparen)
        h, w = img_variant.shape
        center = (w // 2, h // 2)
        
        # Deine bewährte Winkel-Liste
        for angle in [10, -10, 30, 45, 60, -30, -45, -60]: 
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated = cv2.warpAffine(img_variant, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=255)
            
            res_rot = decode(rotated, symbols=[ZBarSymbol.QRCODE])
            if res_rot: return res_rot[0]
            
    return None

# ==========================================
# 3. DIE INTELLIGENTE SUCHE (Smart Tiling)
# ==========================================
def process_roi_smart(roi):
    """
    Kombiniert den ROI-Scan mit einer gezielten Suche im Zentrum und den Ecken.
    Verhindert, dass Codes durch Sliding-Windows "zerschnitten" werden.
    """
    h, w = roi.shape[:2]

    # --- SCHRITT 1: Der "20%-Versuch" (Ganzes Bild) ---
    # Das garantiert, dass wir nicht schlechter als vorher sind!
    if res := robust_decode_func(roi):
        return res, "Full ROI"

    # Wenn der ROI winzig ist, bringt Aufteilen nichts
    if w < 100 or h < 100: return None, None

    # --- SCHRITT 2: Der "Center Crop" (Gegen riesige rote Boxen) ---
    # Wir schneiden die mittleren 60% aus. Oft liegt der Code dort, 
    # aber der Scheibenwischer am Rand stört den Scanner beim Full-Scan.
    cx, cy = w // 2, h // 2
    cw, ch = int(w * 0.6), int(h * 0.6)
    x_start = cx - cw // 2
    y_start = cy - ch // 2
    
    center_crop = roi[y_start:y_start+ch, x_start:x_start+cw]
    if res := robust_decode_func(center_crop):
        return res, "Center Crop"

    # --- SCHRITT 3: Die 4 Ecken (Gegen dezentrale Codes) ---
    # Wir nehmen 4 große Überlappende Bereiche (jeweils 60% des Bildes)
    # Top-Left, Top-Right, Bottom-Left, Bottom-Right
    crops = [
        (roi[0:ch, 0:cw], "Top-Left"),
        (roi[0:ch, w-cw:w], "Top-Right"),
        (roi[h-ch:h, 0:cw], "Bottom-Left"),
        (roi[h-ch:h, w-cw:w], "Bottom-Right")
    ]
    
    for crop_img, name in crops:
        if res := robust_decode_func(crop_img):
            return res, name

    return None, None

# ==========================================
# 4. HAUPTPROGRAMM
# ==========================================
def run_decoding_combined():
    cfg = PostProcessConfig()
    cfg.dir_success.mkdir(parents=True, exist_ok=True)
    cfg.dir_failed.mkdir(parents=True, exist_ok=True)
    
    heatmap_files = list(cfg.heatmap_dir.glob("*_heatmap.png"))
    if not heatmap_files:
        print("❌ Keine Heatmaps gefunden!")
        return

    print(f"📂 Starte 'Combined Force' Analyse (20%-Logik + Smart Crops) von {len(heatmap_files)} Bildern...")
    
    stats_processed = 0
    stats_success = 0
    stats_failed = 0
    stats_codes = 0

    with open(cfg.log_file, 'w', encoding='utf-8') as log:
        log.write("Dateiname;Status;Inhalt;Methode\n") 
        
        for hm_file in heatmap_files:
            stem = hm_file.stem.replace("_heatmap", "")
            
            orig_path = None
            for ext in [".jpg", ".jpeg", ".png", ".JPG", ".PNG"]:
                possible = cfg.original_img_dir / (stem + ext)
                if possible.exists(): orig_path = possible; break
            
            if not orig_path: continue
            stats_processed += 1
            
            img_orig = cv2.imread(str(orig_path))
            img_heatmap = cv2.imread(str(hm_file), cv2.IMREAD_GRAYSCALE)
            
            if img_orig is None: continue
            h_orig, w_orig = img_orig.shape[:2]

            _, thresh = cv2.threshold(img_heatmap, cfg.heatmap_threshold_pixel_val, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            result_img = img_orig.copy()
            image_has_code = False
            found_contents = []

            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                if w < 20 or h < 20: continue 
                
                # ROI Cut
                x_pad = max(0, x - cfg.padding)
                y_pad = max(0, y - cfg.padding)
                w_pad = min(w_orig - x_pad, w + 2*cfg.padding)
                h_pad = min(h_orig - y_pad, h + 2*cfg.padding)
                roi = img_orig[y_pad : y_pad + h_pad, x_pad : x_pad + w_pad]
                
                # --- KOMBINIERTE LOGIK ---
                decoded_obj, method_name = process_roi_smart(roi)

                if decoded_obj:
                    image_has_code = True
                    content = decoded_obj.data.decode("utf-8")
                    found_contents.append(content)
                    stats_codes += 1
                    
                    print(f"   ✅ {stem} [{method_name}]: {content}")
                    cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 255, 0), 4)
                    cv2.putText(result_img, f"OK ({method_name})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    
                    # Wenn wir einen finden, brechen wir die Konturen-Schleife ab (optional),
                    # damit wir keine Duplikate haben.
                    break 
                else:
                    cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 0, 255), 2)

            out_name = f"res_{orig_path.name}"
            if image_has_code:
                stats_success += 1
                cv2.imwrite(str(cfg.dir_success / out_name), result_img)
                for c in found_contents:
                    log.write(f"{orig_path.name};OK;{c};Success\n")
            elif len(contours) > 0:
                stats_failed += 1
                cv2.imwrite(str(cfg.dir_failed / out_name), result_img)
                log.write(f"{orig_path.name};FAILED;-;-\n")

    print("\n" + "="*50)
    print("             ERGEBNIS (COMBINED FORCE)")
    print("="*50)
    if stats_processed > 0:
        rate = (stats_success / stats_processed) * 100
    else: rate = 0
    print(f"Verarbeitet:  {stats_processed}")
    print(f"Erfolg:       {stats_success} ({rate:.1f}%)")
    print(f"Fehlschlag:   {stats_failed}")
    print("-" * 50)
    print(f"Codes:        {stats_codes}")
    print("="*50)

if __name__ == "__main__":
    run_decoding_combined()

📂 Starte 'Combined Force' Analyse (20%-Logik + Smart Crops) von 220 Bildern...
   ✅ Passat_P1_frame40 [Full ROI]: https://testsigma.com/
   ✅ GKLASSE_NEU_P1_frame4050 [Full ROI]: https://testsigma.com/
   ✅ CKlasse_Bilderreihe 5_P_IMG_9517_rot10 [Full ROI]: https://www.h-ka.de/
   ✅ Focus_P2_frame_3000 [Center Crop]: https://testsigma.com/
   ✅ Focus_P2_frame_250 [Center Crop]: https://testsigma.com/
   ✅ GKLASSE_NEU_P1_frame390 [Full ROI]: https://testsigma.com/
   ✅ Passat_P1_frame590 [Full ROI]: https://testsigma.com/
   ✅ Passat_P1_frame1900 [Full ROI]: https://testsigma.com/
   ✅ FocusP3_frame_1590 [Center Crop]: https://testsigma.com/
   ✅ Focus_P2_frame_1450 [Bottom-Left]: https://testsigma.com/
   ✅ Passat_P2_frame2760 [Full ROI]: https://testsigma.com/
   ✅ 20251027_164110 [Center Crop]: https://www.h-ka.de/
   ✅ FocusP4_frame_3670 [Full ROI]: https://testsigma.com/
   ✅ Passat_P1_frame2040 [Bottom-Right]: https://testsigma.com/
   ✅ Focus_P2_frame_2460 [Center Crop]: https://